In [ ]:
import torch
from transformers import pipeline
import nltk

from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

import csv

In [ ]:
# model_path = 'cardiffnlp/twitter-roberta-base-sentiment-latest'
# sentiment_analysis_pipe = pipeline(
#     "sentiment-analysis", 
#     model=model_path, 
#     tokenizer=model_path, 
#     device=0 if torch.cuda.is_available() else -1)

In [ ]:
filename = "data/reviews.csv"
first_column = []
with open(filename, 'r') as csvfile:
  csv_reader = csv.reader(csvfile)
  next(csv_reader, None)
  for row in csv_reader:
    first_column.append(row[0])

In [ ]:
first_column[:1]

In [ ]:
nltk.download('punkt')  # Download sentence tokenizer if not already installed

def paragraphs_to_sentences(paragraphs):
  sentences = []
  for paragraph in paragraphs:
    sentences.extend(nltk.sent_tokenize(paragraph))
  return sentences

sentences = paragraphs_to_sentences(first_column)

In [ ]:
print(len(sentences))
sentences[0]

In [ ]:
texts = sentences

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
with open('output.csv', 'w', newline='') as csvfile:
   fieldnames = ['text', 'positive_score', 'negative_score', 'neutral_score']
   writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
   writer.writeheader()

   for text in texts:
       encoded_input = tokenizer(text, return_tensors='pt')
       output = model(**encoded_input)
       scores = output[0][0].detach().numpy()
       scores = softmax(scores)

       positive_score = scores[config.label2id['positive']]
       negative_score = scores[config.label2id['negative']]
       neutral_score = scores[config.label2id['neutral']]

       writer.writerow({
           'text': text,
           'positive_score': positive_score,
           'negative_score': negative_score,
           'neutral_score': neutral_score
       })